PyTorch Profiler is a tool that allows the collection of the performace metrics during the training and inference. Profiler's context manager API can be used to better understand what model operators are the most expensive, examine their input shapes and stack traces, study device kernel activity and visualize the execution trace. 

https://pytorch.org/docs/master/profiler.html  
https://pytorch.org/tutorials/recipes/recipes/profiler_recipe.html  
https://pytorch.org/tutorials/intermediate/tensorboard_profiler_tutorial.html

#### Steps

1. Import all necessary libraries
2. Instantiate a simple Resnet model
3. Using profiler to analyze execution time
4. Using profiler to analyze memory consumption
5. Using tracing functionality
6. Examining stack traces
7. Visualizating data as flmegraph
8. Using profiler to analyze long-running job

In [4]:
# 1. Import all necessary libraries
import torch
import torchvision.models as models
from torch.profiler import profile, record_function, ProfilerActivity

In [5]:
# 2. Instantiate a simple Resnet model
model = models.resnet18()
inputs = torch.randn(5, 3, 244, 244)

In [6]:
# 3. Using profiler to analyze execution time
with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    with record_function("model_inference"):
        model(inputs)

PyTorch profiler is enableed through the context manager and accepts a number of parameters, some of the most useful are:  
- `activities` - a list of activities to profile:
  - `ProfilerActivity.CPU` - PyTorch operators, TorchScript functions and user-defined code labels
  - `ProfilerActivity.CUDA` - on-device CUDA kernels
- `record-shapes` - whether to record shapes of the operator inputs
- `profile_memory` - whether to report amount of memory consumed by model's Tensors;
- `use_cuda` - whether to measure excution time of CUDA kernels.

Note that we can use `record_function` context manager to label arbitrary code ranges with user provided names(`model_inference` is used as a label in the example above).  
Profiler allows one to check which operators were called during the execution of a code range wrapped with a profiler context manager. If multiple profiler ranges are active at the same time (e.g. in parallel PyTorch threads), each profiling context manager tracks only the operators of its corresponding range. Profiler also automatically profiles the async tasks launched with `torch.jit._fork` and (in case of a backward pass) the backward pass operators launched with `backward()` call. 


In [7]:
print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))

---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                  model_inference         6.65%      18.580ms        99.33%     277.680ms     277.680ms             1  
                     aten::conv2d         0.27%     757.000us        64.32%     179.799ms       8.990ms            20  
                aten::convolution         0.69%       1.938ms        64.05%     179.042ms       8.952ms            20  
               aten::_convolution         0.26%     730.000us        63.35%     177.104ms       8.855ms            20  
         aten::mkldnn_convolution        62.92%     175.907ms        63.09%     176.374ms       8.819ms            20  
                 aten::batch_norm       

Here we see that, as expected, most of the time is spent in convolution (and specifically in `mkldnn_convolution` for PyTorch compiled with MKL-DNN support). Note the difference between self cpu time and cpu time - operators can call other operators, self cpu time exludes time spent in children operator calls, while total cpu time includes it. You can choose to sort by the self cpu time by passing `sort_by="self_cpu_time_total"` into the table call.

To get a finer granularity of results and include operator input shapes, pass `group_by_input_shape=True` (note: this requires running the profiler with `record_shapes=True`):

In [8]:
print(prof.key_averages(group_by_input_shape=True).table(sort_by="cpu_time_total", row_limit=10))

---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  --------------------------------------------------------------------------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls                                                                      Input Shapes  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  --------------------------------------------------------------------------------  
                  model_inference         6.65%      18.580ms        99.33%     277.680ms     277.680ms             1                                                                                []  
                     aten::conv2d         0.01%      35.000us        13.49%      37.711ms       9.428ms             4                             [[5, 64, 61, 61], [64, 64, 3, 3], [], [], [], 

In [12]:
# Profiler can also be used to analyze performance of models executed on GPUs:
model = models.resnet18().cuda()
inputs = torch.randn(5, 3, 224, 224).cuda()

with profile(activities=[
        ProfilerActivity.CPU, ProfilerActivity.CUDA], record_shapes=True) as prof:
    with record_function("model_inference"):
        model(inputs)

print(prof.key_averages().table(sort_by="cuda_time_total", row_limit=10))

---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                  model_inference         0.14%       3.497ms        99.90%        2.566s        2.566s     756.000us         0.03%        2.568s        2.568s             1  
                     aten::conv2d         0.04%       1.127ms        51.68%        1.327s      66.375ms     855.000us         0.03%        1.355s      67.734ms            20  
                aten::convolution         0.03%     726.000us        51.64%        1.326s      66.319ms     341.000us  

In [13]:
# 4. Using profiler to analyze memory consumption
model = models.resnet18()
inputs = torch.randn(5, 3, 224, 224)

with profile(activities=[ProfilerActivity.CPU],
        profile_memory=True, record_shapes=True) as prof:
    model(inputs)

print(prof.key_averages().table(sort_by="self_cpu_memory_usage", row_limit=10))

---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                      aten::empty         0.63%       1.750ms         0.63%       1.750ms      10.938us      94.84 Mb      94.84 Mb           160  
    aten::max_pool2d_with_indices         4.10%      11.424ms         4.10%      11.424ms      11.424ms      11.48 Mb      11.48 Mb             1  
                       aten::mean         0.42%       1.157ms         7.06%      19.646ms     935.524us      28.75 Kb      28.75 Kb            21  
                      aten::addmm         1.17%       3.270ms         1.19%       3.319ms       3.319ms      19.

In [14]:
print(prof.key_averages().table(sort_by="cpu_memory_usage", row_limit=10))

---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                      aten::empty         0.63%       1.750ms         0.63%       1.750ms      10.938us      94.84 Mb      94.84 Mb           160  
                 aten::batch_norm         0.05%     130.000us        26.88%      74.840ms       3.742ms      47.41 Mb           0 b            20  
     aten::_batch_norm_impl_index         0.10%     278.000us        26.84%      74.710ms       3.736ms      47.41 Mb           0 b            20  
          aten::native_batch_norm        20.00%      55.681ms        26.72%      74.389ms       3.719ms      47.

PyTorch profiler can also show the amount of memory (used by the model’s tensors) that was allocated (or released) during the execution of the model’s operators. In the output below, ‘self’ memory corresponds to the memory allocated (released) by the operator, excluding the children calls to the other operators. To enable memory profiling functionality pass `profile_memory=True`.

In [15]:
# 5. Using tracing functionality
# Profiling results can be outputted as a .json trace file:
model = models.resnet18().cuda()
inputs = torch.randn(5, 3, 224, 224).cuda()

with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA]) as prof:
    model(inputs)

prof.export_chrome_trace("trace.json")

In [16]:
# 6. Examing stack traces
with profile(
    activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA],
    with_stack=True,
) as prof:
    model(inputs)

# Print aggregated stats
print(prof.key_averages(group_by_stack_n=5).table(sort_by="self_cuda_time_total", row_limit=2))

---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ---------------------------------------------------------------------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  Source Location                                                              
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ---------------------------------------------------------------------------  
          aten::cudnn_convolution        23.98%       2.592ms        26.48%       2.863ms     143.150us      39.396ms        57.14%      39.532ms       1.977ms            20  lib\runpy.py(75): _run_code                                      

In [17]:
# 7. Visualizing data as a flamegraph
prof.export_stacks("/tmp/profiler_stacks.txt", "self_cuda_time_total")
# We recommend using e.g. Flamegraph tool to generate an interactive SVG:
# git clone https://github.com/brendangregg/FlameGraph
# cd FlameGraph
# ./flamegraph.pl --title "CUDA time" --countname "us." /tmp/profiler_stacks.txt > perf_viz.svg

FileNotFoundError: [Errno 2] No such file or directory: '/tmp/profiler_stacks.txt'

In [ ]:
# 8. Using profiler to analyze long-running jobs
from torch.profiler import schedule

my_schedule = schedule(
    skip_first=10,
    wait=5,
    warmup=1,
    active=3,
    repeat=2)

def trace_handler(p):
    output = p.key_averages().table(sort_by="self_cuda_time_total", row_limit=10)
    print(output)
    p.export_chrome_trace("/tmp/trace_" + str(p.step_num) + ".json")

with profile(
    activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA],
    schedule=torch.profiler.schedule(
        wait=1,
        warmup=1,
        active=2),
    on_trace_ready=trace_handler
) as p:
    for idx in range(8):
        model(inputs)
        p.step()


PyTorch profiler offers an additional API to handle long-running jobs (such as training loops). Tracing all of the execution can be slow and result in very large trace files. To avoid this, use optional arguments:  
- `schedule` - specifies a function that takes an integer argument (step number) as an input and returns an action for the profiler, the best way to use this parameter is to use
- `on_trace_ready` - specifies a function that takes a reference to the profiler as an input and is called by the profiler each time the new trace is ready.

Profiler assumes that the long-running job is composed of steps, numbered starting from zero. The example above defines the following sequence of actions for the profiler:  
1. Parameter `skip_first` tells profiler that it should ignore the first 10 steps (default value of `skip_first` is zero);
2. After the first `skip_first` steps, profiler starts executing profiler cycles;
3. Each cycle consists of three phases:
   1. idling (wait=5 steps), during this phase profiler is not active;
   2. warming up (warmup=1 steps), during this phase profiler starts tracing, but the results are discarded; this phase is used to discard the samples obtained by the profiler at the beginning of the trace since they are usually skewed by an extra overhead;
   3. active tracing (active=3 steps), during this phase profiler traces and records data;
4. An optional repeat parameter specifies an upper bound on the number of cycles. By default (zero value), profiler will execute cycles as long as the job runs.
   
Thus, in the example above, profiler will skip the first 15 steps, spend the next step on the warm up, actively record the next 3 steps, skip another 5 steps, spend the next step on the warm up, actively record another 3 steps. Since the repeat=2 parameter value is specified, the profiler will stop the recording after the first two cycles.

At the end of each cycle profiler calls the specified on_trace_ready function and passes itself as an argument. This function is used to process the new trace - either by obtaining the table output or by saving the output on disk as a trace file.

To send the signal to the profiler that the next step has started, call prof.step() function. The current profiler step is stored in prof.step_num.